# **Clasterización**

En este archivo realizaremos una clasterización que verifica los siguientes puntos:
- Aprendizaje No-Supervisado en Python.
- Ingeniería de variables (Feature engineering).
- Centroid-based Clustering (K-Means , Mean-Shift & Mini-Batch K-Means).
- Density-based clustering (DBSCAN, OPTICS).
- Distribution-based clustering (GMM).
- Hierarchical clustering (Agglomerative Clustering).

Primero importamos todas las librerías que usaremos y las instalamos en caso de ser necesario.

**¿Cómo hacemos que el código sea de aprendizaje no supervisado?**

El aprendizaje no supervisado se aplica en este código a través de los algoritmos de clustering que se utilizan para agrupar los datos en grupos o clusters sin la necesidad de tener etiquetas de clase predefinidas. 

Estos algoritmos son ejemplos de técnicas de aprendizaje no supervisado que se utilizan para encontrar patrones y estructuras subyacentes en los datos sin necesidad de información previa sobre las clases a las que pertenecen los datos. El objetivo es agrupar los datos en grupos similares o "clústeres" basándose únicamente en la similitud entre las muestras. Cada algoritmo tiene su propio enfoque para realizar esta tarea, como la minimización de la distancia intra-cluster en el caso de K-Means o la detección de densidades locales en el caso de DBSCAN.

In [24]:
# Importamos las librerías necesarias

# Pandas para manipulación de datos
import pandas as pd

# Codificación de etiquetas y escalado de características
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Algoritmos de clustering
from sklearn.cluster import KMeans, MeanShift, DBSCAN, OPTICS, AgglomerativeClustering

# Algoritmo de mezcla gaussiana para clustering
from sklearn.mixture import GaussianMixture

# Métricas de evaluación de clustering
from sklearn.metrics import silhouette_score, davies_bouldin_score

El próximo paso es cargar los datos limpios.

In [9]:
datos = pd.read_csv('../../data/partidos_limpio.csv')
datos.head()

,Season,Round,Day,Date,Results,Home,Country (Home),Points (Home),Score (Home),Score (Away),...,MP_away,Starts_away,Gls_away,Ast_away,G+A_away,G-PK_away,PK_away,PKatt_away,CrdY_away,CrdR_away
0,2023-2024,Round of 16,Tue,2024-02-13,A,RB Leipzig,Germany,88.736698,0,1,...,10.0,110.0,20.0,17.0,37.0,20.0,0.0,1.0,18.0,0.0
1,2023-2024,Round of 16,Tue,2024-02-13,A,FC Copenhagen,Denmark,80.431647,1,3,...,10.0,110.0,28.0,20.0,48.0,25.0,3.0,3.0,10.0,0.0
2,2023-2024,Round of 16,Wed,2024-02-14,H,Paris S-G,France,114.334580,2,0,...,8.0,88.0,8.0,5.0,13.0,8.0,0.0,1.0,18.0,0.0
3,2023-2024,Round of 16,Wed,2024-02-14,H,Lazio,Italy,99.943311,1,0,...,10.0,110.0,18.0,14.0,32.0,16.0,2.0,2.0,13.0,1.0
4,2023-2024,Round of 16,Tue,2024-02-20,D,PSV Eindhoven,The Netherlands,98.784903,1,1,...,10.0,110.0,15.0,12.0,27.0,14.0,1.0,1.0,16.0,0.0


Ahora comenzamos con un preprocesamiento de los datos que puede considerarse como una forma básica de ingeniería de variables. En nuestro caso es el manejo de valores faltantes, eliminación de columnas irrelevantes y el escalado de características.

In [10]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          598 non-null    object 
 1   Round           598 non-null    object 
 2   Day             598 non-null    object 
 3   Date            598 non-null    object 
 4   Results         598 non-null    object 
 5   Home            598 non-null    object 
 6   Country (Home)  598 non-null    object 
 7   Points (Home)   598 non-null    float64
 8   Score (Home)    598 non-null    int64  
 9   Score (Away)    598 non-null    int64  
 10  Points (Away)   598 non-null    float64
 11  Country (Away)  598 non-null    object 
 12  Away            598 non-null    object 
 13  Venue           598 non-null    object 
 14  Referee         598 non-null    object 
 15  # Pl_home       538 non-null    float64
 16  Age_home        538 non-null    float64
 17  MP_home         538 non-null    flo

Vemos que todavía tenemos algunas filas nulas. Al limpiar los datos no nos importaba tener algunas filas nulas, pero para hacer la clasterización es muy importante no contar con ningún dato de este tipo.

In [11]:
# Eliminamos las filas que contienen valores nulos
datos = datos.dropna()

# Vemos que se ha hecho el cambio correctamente
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538 entries, 0 to 597
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          538 non-null    object 
 1   Round           538 non-null    object 
 2   Day             538 non-null    object 
 3   Date            538 non-null    object 
 4   Results         538 non-null    object 
 5   Home            538 non-null    object 
 6   Country (Home)  538 non-null    object 
 7   Points (Home)   538 non-null    float64
 8   Score (Home)    538 non-null    int64  
 9   Score (Away)    538 non-null    int64  
 10  Points (Away)   538 non-null    float64
 11  Country (Away)  538 non-null    object 
 12  Away            538 non-null    object 
 13  Venue           538 non-null    object 
 14  Referee         538 non-null    object 
 15  # Pl_home       538 non-null    float64
 16  Age_home        538 non-null    float64
 17  MP_home         538 non-null    flo

Observamos que hay algunas variables categóricas que podríamos pasar a numéricas, pues nos podrían ayudar en nuestra predicción posteriormente.

In [12]:
# Columnas a modificar
cols = ['Season', 'Round', 'Day', 'Results', 'Home', 'Away', 'Country (Home)', 'Country (Away)', 'Venue', 'Referee']

# Inicializamos el label encoder
label_encoder = LabelEncoder()

# Creamos un diccionario para guardar los mapeos
mapping = {}

# Iteramos sobre las columnas y las transformamos
for col in cols:
    # Concatenamos los valores necesarios
    if col in ['Home', 'Away']:
        if 'Squad' not in mapping:
            name = 'Squad'
            squad = pd.concat([datos['Home'], datos['Away']])
            label_encoder.fit(squad)      
    elif col in ['Country (Home)', 'Country (Away)']:
        if 'Country' not in mapping:
            name = 'Country'
            country = pd.concat([datos['Country (Home)'], datos['Country (Away)']])
            label_encoder.fit(country)
    else:
        name = col
        label_encoder.fit(datos[col])
    
    # Transformamos los valores 
    datos[col] = label_encoder.transform(datos[col])
    
    # Creamos un mapeo de los valores
    mapping[name] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Transformamos la columna 'Date' a datetime
datos['Date'] = pd.to_datetime(datos['Date'])

# Separar la fecha en año, mes y día
datos['Year'] = datos['Date'].dt.year
datos['Month'] = datos['Date'].dt.month
datos['Number Day'] = datos['Date'].dt.day # Lo llamamos 'Number Day' para evitar confusiones con la columna 'Day' que ya existe

# Eliminamos la columna 'Date'
datos.drop('Date', axis=1, inplace=True)

# Verificamos los cambios
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538 entries, 0 to 597
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Season          538 non-null    int32  
 1   Round           538 non-null    int32  
 2   Day             538 non-null    int32  
 3   Results         538 non-null    int32  
 4   Home            538 non-null    int32  
 5   Country (Home)  538 non-null    int32  
 6   Points (Home)   538 non-null    float64
 7   Score (Home)    538 non-null    int64  
 8   Score (Away)    538 non-null    int64  
 9   Points (Away)   538 non-null    float64
 10  Country (Away)  538 non-null    int32  
 11  Away            538 non-null    int32  
 12  Venue           538 non-null    int32  
 13  Referee         538 non-null    int32  
 14  # Pl_home       538 non-null    float64
 15  Age_home        538 non-null    float64
 16  MP_home         538 non-null    float64
 17  Starts_home     538 non-null    flo

Último paso de la ingeniería de variables, realizamos un escalado de características.

In [13]:
# Escalamos los datos para mejorar la efectividad de los algoritmos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(datos)

Aplicamos los algoritmos de clustering.

In [14]:
algorithms = {
    "KMeans": KMeans(n_clusters=3),
    "MeanShift": MeanShift(),
    "DBSCAN": DBSCAN(eps=0.5, min_samples=5),
    "OPTICS": OPTICS(),
    "GMM": GaussianMixture(n_components=3),
    "AgglomerativeClustering": AgglomerativeClustering(n_clusters=3)
}

Iteramos sobre varios algoritmos de clustering, aplicamos cada algoritmo al conjunto de datos, contamos el número de clusters encontrados por cada algoritmo y luego imprimimos esta información para cada algoritmo. Esto permite comparar cómo cada algoritmo agrupa los datos y cuántos clusters encuentra.

In [15]:
for name, algorithm in algorithms.items():
    cluster_labels = algorithm.fit_predict(X_scaled)
    if cluster_labels is not None:
        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        print(f"{name}: {n_clusters} clusters")
    else:
        print(f"Error: {name} algorithm returned None")

KMeans: 3 clusters
MeanShift: 3 clusters
DBSCAN: 0 clusters
OPTICS: 1 clusters
GMM: 3 clusters
AgglomerativeClustering: 3 clusters


### **Evaluación** 

La superposición y la separación de los clústeres son aspectos importantes a considerar al evaluar la calidad de un algoritmo de clustering y al interpretar los resultados del mismo.  Si los clústeres están claramente separados, es más fácil interpretar y asignar significado a cada clúster. Por otro lado, clústeres superpuestos pueden hacer que sea más difícil generalizar y aplicar los resultados del clustering a nuevos datos. Si los clústeres se superponen significativamente, es posible que las fronteras entre los grupos no estén bien definidas, lo que puede llevar a una interpretación menos clara de los resultados y a decisiones menos confiables basadas en ellos.

Los coeficientes de silueta y la medida de Davies-Bouldin son métricas comunes utilizadas para evaluar la calidad de los clústeres obtenidos a partir de un algoritmo de clustering.

- **Coeficiente de Silueta**: Este coeficiente mide qué tan similar es un punto a su propio clúster (cohesión) en comparación con otros clústeres (separación). El valor del coeficiente de silueta varía entre -1 y 1. Un valor más alto indica que los puntos están bien agrupados dentro de sus clústeres y están separados de otros clústeres. Un valor cercano a 0 indica superposición de clústeres.

- **Medida de Davies-Bouldin**: Esta medida compara la dispersión dentro de los clústeres con la dispersión entre los clústeres. Un valor más bajo de la medida de Davies-Bouldin indica una mejor partición (clustering). Los valores más bajos cercanos a cero indican clústeres densos y bien separados.

In [21]:
# Calcular el coeficiente de silueta
silhouette = silhouette_score(X_scaled, cluster_labels)
print(f"Coeficiente de Silueta: {silhouette}")

# Calcular la medida de Davies-Bouldin
davies_bouldin = davies_bouldin_score(X_scaled, cluster_labels)
print(f"Medida de Davies-Bouldin: {davies_bouldin}")

Coeficiente de Silueta: 0.08176731903394704
Medida de Davies-Bouldin: 2.770795707086679


El Coeficiente de Silueta es de aproximadamente 0.08, lo que sugiere una separación moderada entre los clústeres y una cierta superposición. Además, la Medida de Davies-Bouldin es de aproximadamente 2.77, lo que sugiere una partición moderadamente buena de los datos en clústeres.Podemos concluir que los valores de nuestras métricas indican que los clústeres no están perfectamente definidos y podrían haber áreas de superposición entre ellos. 

En el contexto del fútbol, donde tantos factores pueden influir en los resultados de un partido, como la forma del equipo, el clima o las decisiones del árbitro, es natural que los datos sean un poco desordenados. Del mismo modo, en el clustering, a veces los grupos pueden superponerse un poco porque los datos son complejos y no siempre se ajustan perfectamente a categorías claras. 